In [6]:
import pymongo
import pprint
from pymongo import MongoClient

Pre-step

In [7]:


client = MongoClient('localhost', 27017)
db = client.Northwind # Get the database
customer = db['customers'] # Get the collection
orders = db['orders']
orderdetails = db['order-details'] 
products = db['products'] 
regions=db['regions']
db.collection_names()


[u'suppliers',
 u'territories',
 u'regions',
 u'northwind',
 u'categories',
 u'products',
 u'employee-territories',
 u'orders',
 u'customers',
 u'shippers',
 u'order-details']

Exercise 1

In [9]:

# initial order list
orderIDl = []
#initial the productID list()
prol = [] 

#forloops 1 : in order to get the OrderID which corresponding to the given CustomerID
for o in db.orders.find({'CustomerID': 'ALFKI'}).distinct('OrderID'):
    orderIDl.append(o)
print 'Order ID :',orderIDl

#forloops 2: in order to get the productID by using iterated orderID.
for i in range(len(orderIDl)):
    #print orderl[i]
    for productID in orderdetails.find({'OrderID': orderIDl[i]}).distinct('ProductID'):
        prol.append(productID)
        
#for loop 3: in order to get the Product name by using iterated ProductID.
for j in range(len(prol)):
        #print products.find({'ProductID': prol[p]}).distinct('ProductID')
    for product in products.find({'ProductID': prol[j]}).distinct('ProductName'):
        print product,'Product ID:',prol[j]



Order ID : [10643, 10692, 10702, 10835, 10952, 11011]
++++++++++++++++++++++++++++++++++++++++++++++++
Rössle Sauerkraut Product ID: 28
Chartreuse verte Product ID: 39
Spegesild Product ID: 46
Vegie-spread Product ID: 63
Aniseed Syrup Product ID: 3
Lakkalikööri Product ID: 76
Raclette Courdavault Product ID: 59
Original Frankfurter grüne Soße Product ID: 77
Grandma's Boysenberry Spread Product ID: 6
Rössle Sauerkraut Product ID: 28
Escargots de Bourgogne Product ID: 58
Flotemysost Product ID: 71


Exercise 2

In [16]:


orderpipe = [{ "$match" : { "CustomerID": "ALFKI" } }, # $match uses standard MongoDB queries.
        
             { "$lookup" :  { "from" : "order-details", 
            "localField" : "OrderID","foreignField" : "OrderID","as" : "orders_for_customer"}},
            
            { "$lookup" : {"from" : "products","localField" : "orders_for_customer.ProductID",
            "foreignField" : "ProductID","as" : "products_for_order"}}, 
            
            { "$project" : {"CustomerID" : "$CustomerID", "OrderID" : "$OrderID",
            "ProductID" : "$products_for_order.ProductID","NumberOfOrders" :
            {"$size" : "$orders_for_customer"},"_id" : 0}},
            
            { "$match" : { "NumberOfOrders" : {"$gt" : 1}}}] # NumberofOrders > 1


pprint.pprint(list(orders.aggregate(orderpipe)))

[{u'CustomerID': u'ALFKI',
  u'NumberOfOrders': 3,
  u'OrderID': 10643,
  u'ProductID': [28, 39, 46]},
 {u'CustomerID': u'ALFKI',
  u'NumberOfOrders': 2,
  u'OrderID': 10702,
  u'ProductID': [3, 76]},
 {u'CustomerID': u'ALFKI',
  u'NumberOfOrders': 2,
  u'OrderID': 10835,
  u'ProductID': [59, 77]},
 {u'CustomerID': u'ALFKI',
  u'NumberOfOrders': 2,
  u'OrderID': 10952,
  u'ProductID': [6, 28]},
 {u'CustomerID': u'ALFKI',
  u'NumberOfOrders': 2,
  u'OrderID': 11011,
  u'ProductID': [58, 71]}]


Exercise 3

In [17]:

dicti = db.products.find_one() # Get the first content of product 
pprint.pprint(dicti) # print pretty to show
pID = dicti['CategoryID'] #extract the CategoryID
sID = dicti['SupplierID'] #Extract the SupplierID

# Print them to check  
print "The category ID of first product is: ",pID
print "The SupplierID is:",sID

#Find the infromation of products with obtained CategoryID and SupplierID
pro=products.find({'CategoryID':pID,'SupplierID':sID}).distinct('ProductName') 

#for loop to print ProductName(s)
for pName in pro:
    print "Name of product with productID:1,supplyID:1 are:",(pName)

{u'CategoryID': 1,
 u'Discontinued': 0,
 u'ProductID': 1,
 u'ProductName': u'Chai',
 u'QuantityPerUnit': u'10 boxes x 20 bags',
 u'ReorderLevel': 10,
 u'SupplierID': 1,
 u'UnitPrice': 18.0,
 u'UnitsInStock': 39,
 u'UnitsOnOrder': 0,
 u'_id': ObjectId('59e360517f17b9af0c4d1fc2')}
The category ID of first product is:  1
The SupplierID is: 1
Name of product with productID:1,supplyID:1 are: Chai
Name of product with productID:1,supplyID:1 are: Chang
